## __Webbapplikation för Bostadsrättsförening__
___

#### DD1334, Databasteknologi
Oskar Edén Wallberg, 20001114-6016, oskarew@ug.kth.se, @codezart44 (GitHub) <br>
Alexander Emrik Karolin, 200221202-XXXX, alex??@ug.kth.se, @AlexEmrik (GitHub) <br>




![WebbsiteInterface](./images/interface.png)

### Beskrivning
___
Detta projekt fokuserar på säkerhet och inkorporerar krypteringstekniker i form av hashnings (blowfish-cyphern) samt saltning för att motverka "brute-force" attacker. <br>
Projektet består av två stora delprojekt:
- __Backend__ (_Server side_): Skriven i python och uppsatt med biblioteket Flask. Backend är i direkt kontakt med batabasen (SQLite3) och kommer genom API-anrop att kommunicera, extrahera och addera information till och från databasen på standardiserade sätt opåverkade av klientsidan. Där ingår även validering av datan och säkerhet från illvilliga användare. Bland annat ingår metoder (API-endpoints) för att registrera samt logga in från klientsidan. 
Huvudsakligt kod finns under: ```./backend/flask_server/routes/auth.py```

- __Frontend__ (_Client side_): Skriven i jsx (JS XML) med js biblioteket React (industrinära och relevant på arbetsmarknaden). Användargränssnittet utgörs av en webbsida (lokalt hostad) med interface för inloggning och registrering. Vid inloggning samt registrering skickas anrop med formulärdata till API-ändpunkterna i backend som tar emot POST request data, extraherar relevant information och ger en lämplig response. 
Huvudsaklig kod finns under: ```./frontend/src/components/login/Login.jsx```

### Krav för kursen
___
#### __Moment 1, HEM1__:
_Skapa ditt eget projekt baserat på ditt eget intresse som på något sätt har att göra med att lagra/hämta data._
- Förklara funktioner och struktur i ett databashanteringssystem
- Diskutera fördelar och nackdelar med olika databasmodeller
- Modellera och strukturera data enligt faktiska begränsningar
- Presentera självständigt förvärvad nödvändig kunskap muntligt och skriftligt.

#### __Moment 2, LABA__:
_Teknisk demonstration - visa att du har bekantat dig med koncepten och reflektera över dem._
- Använd ett query språk för att formulera frågor och beskriva den matematiska grunden för query
- Skriva inbäddade SQL-queries i ett tredje generationens programmeringsspråk
- Förklara konsekvenserna av olika begränsningar på databasstrukturen

//OBS
___
_All kod återfinns i foldrarna i denna projektmapp, både för proejktmoment (HEM1) och labbmoment (LABA). Denna fil är främst till för att vägleda och demonstrera projektet översiktligt._

### DEMO
___
För att kunna köra demo krävs följande setup:
1. __Installera bibliotek och moduler (dependencies)__
    - Navigera till backendmappen och installera nödvändiga bibliotek
        - `cd backend`
        - `pip install -r requirements.txt`
        - Vänta tills installationen är klar, kan ta någon minut. 
    - Navigera sedan till frontendmappen och utför motsvarande
        - `cd ../frontend`
        - `npm install`
        - Vänta tills installationen är klar (detta installeras lokalt för projektet)
2. __Starta Backend servern__
    - Kör filen app.py antingen genom att navigera till den och klicka på körknappen alt:
        - `cd ..`         #navigera tillbaka till root dir från frontend om så behövs.
        - `python3 backend/flask_server/app.py`
        - I terminalen ska det synas att servern är igång: ```Running on http://127.0.0.1:5000``` (ignorera varning om development mode)

#### Nu demo körbart (så länge servern är igång)

3. __Kör riktiga version med frontend användargränssnitt__
    - Navigera till frontend och starta klientserver
        - `cd frontend`
        - `npm run dev`
        - Det ska nu synas: `Local:   http://localhost:5173/`
        - Följ url och testa att skapa ett konto och sedan logga in. 


In [31]:
# imports
import requests
import pandas as pd

print(f'pd=={pd.__version__}')

pd==2.1.0


#### Testa registrering
___

In [33]:
# testa ändra registreringsdatan för att lägga till nya användare
# Kan endast lägga till användare med unik email

url_register = 'http://127.0.0.1:5000/auth/register'

register_data = {
    'first_name': 'Michael',
    'last_name': 'Jackson',
    'email': 'jichael@mackson.se',
    'password': 'heehee'
}

response = requests.post(url=url_register, json=register_data)

print(response.json()['message'])
print(response.status_code)

Email already in use
200


#### Testa inloggning
___

In [34]:
# registrera först en användare (kom ihåg att minnas lösenordet!)
# nu kan du loggan in och börja boka tvättider (se vad som händer om du skriver in fel lösenord eller email)

url_login = 'http://127.0.0.1:5000/auth/login'

login_data = {
    'email': 'jichael@mackson.se',
    'password': 'heehee'
}

response = requests.post(url=url_login, json=login_data)

print(response.json()['message'])
print(response.status_code)

Login success.
200


#### SQL embedded i funktioner av tredje generationens programmeringspråk (python)
___
Exempel:

```
def check_email_exists(email, con): 
    sql_query = f"""
    --sql
    SELECT EXISTS(SELECT 1 FROM users WHERE email = ?) AS email_exists
    ;
    """
    df = pd.read_sql_query(sql=sql_query, con=con, params=(email, ))
    emailExists = (df['email_exists'].values[0] == 1)
    return emailExists
```


```
def authorize_login(email, password, con):
    sql_query = f"""
    --sql
    SELECT 
        [user_id],
        [password]
    FROM users
    WHERE email = ?
    ;
    """
    result = pd.read_sql_query(sql_query, con, params=(email,))

    user_data = result.values.ravel().tolist()
    pw_hash: str = user_data.pop(-1)
    authorized = bcrypt.checkpw(password=password.encode('utf-8'), hashed_password=pw_hash.encode('utf-8'))
    return authorized
```

### Diskussion om fördelar och begränsaningar med "Relational Database Management Systems" (RDBMS)
___

__Fördelar__:
- Data lagras strukturerade och förutsägbara sätt vilket gör den lätt åtkomlig och unvdiker korrupterad eller skadlig data
- Tabulär data kan indexeras vilket gör sökningar (queries) mycket effektiva
- Struktureringen gör datan lättolkad för människor som kan behöva arbete med stora mängder data och utvinna insikt i mönster odyl.
- Underlättar datamigrering och exportering till andra filformat som följer liknande struktur som .csv el. .xlsx.
- Genom unicka nycklar (primary keys & foreign keys) kan relationer mellan olika tabeller sättas upp
    - Detta möjliggör för mer komplex funktionalitet (t.ex SQL triggers som skulle radera all relaterad data när en kund tas bort från systemet)

_Allt som allt gör det data mer underhållbar, skalbar, förflyttningsbar och tolkbar._

__Begränsningar__:
- Data av större format som audiofiler, videofiler passar inte alltid bra att lagra i RDBMS, där kan istället data lakes vara lämpligare.
- Det tabulära formatet kräver ibland upprepning av värden och kan bli verbosa. Detta kan orsaka onödiga kostander på t.ex cloud plattformar (AWS, GCP, MS Azure).
- Stora databaser kan bli komplexa och skapa onödig komplexitet speciellt med många olika relationer mellan tabeller / scheman.
- RDBMS är optimerade för read-funktioner och write-funktioner kan därmed vara tidskrävande. 
- RDBMS är inte optimala för hieraktisk data, där passar JSON formatet bättre. 
